# Dogs breeds

https://youtu.be/JNxcznsrRb8?t=1h31m8s

In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [21]:
torch.cuda.set_device(0)

In [22]:
PATH = "/home/embsysa/fastai/pytorch_classifiers-master/data/"
sz = 224
arch = resnext50
bs = 24

## Initial exploration

In [33]:
import os
data_path = "/home/embsysa/fastai/pytorch_classifiers-master/data"
os.chdir(data_path)

In [53]:
!ls {PATH}

label_csv = 'Major_Graphics_labels.csv'
label_df = pd.read_csv(label_csv)


 Backup			       Major_Graphics_labels.csv	 tmp
'change file type.ipynb'       Major_Graphics_labels_final.csv	 train
 Major_Background_labels.csv   test


In [54]:
images = [each.split(".jpg")[0] for each in os.listdir(data_path+"/train")]

In [57]:
label_df = label_df[label_df.Image_index.isin(images)]
label_df.to_csv('Major_Graphics_labels_final.csv', index=False)

In [58]:
label_csv = f'{PATH}Major_Graphics_labels_final.csv'
n = len(list(open(label_csv))) - 1 # header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [65]:

label_df.head()
label_df.columns = ['id', 'color']

label_df.shape

label_df.pivot_table(index="color", aggfunc=len).sort_values('id', ascending=False)

import numpy as np
np.max(val_idxs)

tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}Major_Graphics_labels_final.csv', test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

fn = PATH + data.trn_ds.fnames[0]; fn

data.trn_ds.fnames[0]

img = PIL.Image.open(fn); img

img.size

(512, 512)

In [66]:
data.trn_ds

In [67]:
size_d = {k: PIL.Image.open(PATH + k).size for k in data.trn_ds.fnames}

In [68]:
row_sz, col_sz = list(zip(*size_d.values()))

In [69]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

## Initial model

In [72]:
def get_data(sz, bs): # sz: image size, bs: batch size
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}Major_Graphics_labels_final.csv', test_name='test',
                                       val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/13
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/37
    return data if sz > 300 else data.resize(256, 'tmp') # Reading the jpgs and resizing is slow for big images, so resizing them all to 340 first saves time

#Source:   
#    def resize(self, targ, new_path):
#        new_ds = []
#        dls = [self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl]
#        if self.test_dl: dls += [self.test_dl, self.test_aug_dl]
#        else: dls += [None,None]
#        t = tqdm_notebook(dls)
#        for dl in t: new_ds.append(self.resized(dl, targ, new_path))
#        t.close()
#        return self.__class__(new_ds[0].path, new_ds, self.bs, self.num_workers, self.classes)
#File:      ~/fastai/courses/dl1/fastai/dataset.py

### Precompute

In [73]:
data = get_data(sz, bs)

In [74]:
#arch

In [75]:
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [76]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [0.001, 0.01, 0.1]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.


learn.precompute = False

In [77]:
learn.fit(1e-2, 50)

epoch      trn_loss   val_loss   accuracy                   
    0      2.201785   4.190137   0.231678  
    1      2.156174   2.338907   0.318361                   
    2      2.078139   2.009785   0.352246                   
    3      2.003353   2.203444   0.281324                   
    4      1.94139    2.681439   0.337273                   
    5      1.950597   2.199478   0.334909                   
    6      1.914647   1.952432   0.371158                   
    7      1.892073   1.928691   0.351458                   
    8      1.872873   1.957842   0.35461                    
    9      1.869886   1.826498   0.344366                   
    10     1.824915   2.024678   0.319937                   
    11     1.852886   1.830355   0.34279                    
    12     1.823404   1.904265   0.37037                    
    13     1.799203   1.865767   0.35461                    
    14     1.80889    1.816895   0.368006                   
    15     1.811138   1.831967   0.37037 

[1.7998407777601382, 0.34278960867694647]

### Augment

In [15]:
from sklearn import metrics

In [16]:
data = get_data(sz, bs)

In [17]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [18]:
learn.fit(1e-2, 2)

epoch      trn_loss   val_loss   accuracy                   
    0      1.973155   1.776888   0.388388  
    1      1.850747   1.731229   0.398899                   


[1.7312292469156396, 0.3988989099815443]

In [19]:
learn.precompute = False

In [20]:
learn.fit(1e-2, 5, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                   
    0      1.69799    1.645528   0.441441  
    1      1.64686    1.640117   0.441942                   
    2      1.633071   1.630674   0.449449                   
 25%|██▌       | 84/334 [00:11<00:50,  4.98it/s, loss=1.62]

KeyboardInterrupt: 

In [ ]:
learn.save('224_pre')

In [ ]:
learn.load('224_pre')

## Increase size

In [21]:
# Starting training on small images for a few epochs, then switching to bigger images, and continuing training is an amazingly effective way to avoid overfitting.

# http://forums.fast.ai/t/planet-classification-challenge/7824/96
# set_data doesn’t change the model at all. It just gives it new data to train with.
learn.set_data(get_data(299, bs)) 
learn.freeze()

#Source:   
#    def set_data(self, data, precompute=False):
#        super().set_data(data)
#        if precompute:
#            self.unfreeze()
#            self.save_fc1()
#            self.freeze()
#            self.precompute = True
#        else:
#            self.freeze()
#File:      ~/fastai/courses/dl1/fastai/conv_learner.py

In [22]:
learn.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 299, 299]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('trainable', False),
                           ('nb_params', tensor(9408))])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('trainable', False),
                           ('nb_params', tensor(128))])),
             ('ReLU-3',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('nb_params', 0)])),
             ('MaxPool2d-4',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 75, 75]),
                           ('nb_params', 0)])),
             ('Conv2d-5',
              OrderedDict([('input_shape

In [23]:
learn.fit(1e-2, 3, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                   
    0      1.60298    1.608781   0.452452  
    1      1.567467   1.593151   0.464965                   
    2      1.536843   1.592957   0.472973                   


[1.5929573065525777, 0.47297298603945664]

Validation loss is much lower than training loss. This is a sign of underfitting. Cycle_len=1 may be too short. Let's set cycle_mult=2 to find better parameter.

In [29]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [0, 0, 0.1]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.

In [30]:
# When you are under fitting, it means cycle_len=1 is too short (learning rate is getting reset before it had the chance to zoom in properly).
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2) # 1+2+4 = 7 epochs

  0%|          | 0/334 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory

Training loss and validation loss are getting closer and smaller. We are on right track.

In [ ]:
log_preds, y = learn.TTA() # (5, 2044, 120), (2044,)
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

In [ ]:
len(data.val_ds.y), data.val_ds.y[:5]

In [ ]:
learn.save('299_pre')

In [ ]:
learn.load('299_pre')

In [ ]:
learn.fit(1e-2, 1, cycle_len=2) # 1+1 = 2 epochs

In [ ]:
learn.save('299_pre')

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

This dataset is so similar to ImageNet dataset. Training convolution layers doesn't help much. We are not going to unfreeze.

## Create submission

https://youtu.be/9C06ZPF8Uuc?t=1905

In [ ]:
data.classes

In [ ]:
data.test_ds.fnames

In [ ]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrcs.log_loss(y, probs) # This does not make sense since test dataset has no labels

In [ ]:
probs.shape # (n_images, n_classes)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [ ]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]
fn

In [ ]:
Image.open(PATH + fn).resize((150, 150))

In [ ]:
# Method 1.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
learn.data.classes[np.argmax(preds)]

In [ ]:
# Method 2.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
im = val_tfms(open_image(PATH + fn)) # open_image() returns numpy.ndarray
preds = learn.predict_array(im[None])
np.argmax(preds)